# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [14]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/StriderKeni/Documents/Data_Science/Data_Engineering/Projects/cassandra-ETL


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [2]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    # the number below include header
    print(sum(1 for line in f))

6821


# Part II. Modeling NoSQL Database

## event_datafile_new.csv columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of the file <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [60]:
# Creating a cluster using local machine (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # Creating a session variable to  establish connection and begin executing queries
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [61]:
# Keyspace for NoSQL database
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [62]:
# Set KEYSPACE to the keyspace specified above
## This allow to start using and executing queries to the keyspace
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## Queries based on needs of the Analytic Team

 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Creating Data Model based on Queries Above

In [47]:
# Drop table statement to be sure that is not created before. 
try:
    session.execute('DROP TABLE IF EXISTS music_library_first')
    session.execute('DROP TABLE IF EXISTS music_library_second')
    session.execute('DROP TABLE IF EXISTS music_library_third')
except Exception as e:
    print(e)

## First Query

- Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [64]:
# CREATE TABLE statement for first table based on first query.
query_1 = "CREATE TABLE IF NOT EXISTS music_library_first"
query_1 = query_1 + "(session_id int, item_session int, user_id int, first_name text, last_name text, artist text, song text, length decimal, PRIMARY KEY((session_id), item_session))"

try:
    session.execute(query_1)
except Exception as e:
    print(e)                    

In [65]:
from decimal import Decimal 
file = 'event_datafile_new.csv'

# Open the csv file with data and iterate into every row.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert statement for first table
        query = "INSERT INTO music_library_first (session_id, item_session, user_id, first_name, last_name, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        # Execute insert iterating into every row of the CSV File
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[1], line[4], line[0], line[9], Decimal(line[5])))
        

#### SELECT to verify that the data have been inserted into each table

In [50]:
## Select statement to verify if the data was inserted into the table
try:
    rows = session.execute("SELECT * FROM music_library_first")
except Exception as e:
    print(e)
    
print("{} rows inserted".format(len(list(rows))))

6820 rows inserted


In [51]:
# Select First Query
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
try:
    results_1 = session.execute("SELECT artist, song, length FROM music_library_first \
    WHERE session_id = 338 and item_session = 4")
except Exception as e:
    print(e)
    
for row in results_1:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Second Query

- Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [52]:
# CREATE STATEMENT for second table based on second query.
query_2 = "CREATE TABLE IF NOT EXISTS music_library_second"
query_2 = query_2 + "(user_id int, session_id int, item_session int, first_name text, last_name text, artist text, song text, PRIMARY KEY((user_id), session_id, item_session))"

try:
    session.execute(query_2)
except Exception as e:
    print(e)         

In [53]:
# Insert into second table 

# Open the csv file with data and iterate into every row.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert statement for second table 
        query = "INSERT INTO music_library_second (user_id, session_id, item_session, first_name, last_name, artist, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Execute insert iterating into every row of the CSV File
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [54]:
# SELECT for second query
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

try:
    results_2 = session.execute("SELECT artist, song, first_name, last_name, user_id, session_id, item_session \
    FROM music_library_second WHERE user_id = 10 and session_id = 182")
except Exception as e:
    print(e)
    
for row in results_2:
    print(row.artist, row.song, row.first_name, row.last_name, row.user_id, row.session_id, row.item_session)

Down To The Bone Keep On Keepin' On Sylvie Cruz 10 182 0
Three Drives Greece 2000 Sylvie Cruz 10 182 1
Sebastien Tellier Kilometer Sylvie Cruz 10 182 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 10 182 3


## Third Query 

- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [55]:
# CREATE STATEMENT for third table based on third query.
query_3 = "CREATE TABLE music_library_third"
query_3 = query_3 + "(song text, user_id int, first_name text, last_name text, artist text, PRIMARY KEY((song), user_id, first_name, last_name))"

# We are going to use SONG and USER_ID as Primary Key and Clustering Key, this will allow the database to update the values
## which may be ok based on the statement of the query, because the team is asking about a list of users (name and last name)
### and not about how many times that song was listened. That's why I decided to create the table in that order.
#### I added some clustering columns like first_name and last_name to order the output data by name and last name
try:
    session.execute(query_3)
except Exception as e:
    print(e)        

In [56]:
# Insert into third table 

# Open the csv file with data and iterate into every row.
with open(file, encoding='utf-8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    for line in csvreader:
        # Insert statement 
        query = "INSERT INTO music_library_third (song, user_id, first_name, last_name, artist)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Execute insert into third table
        session.execute(query, (line[9], int(line[10]), line[1], line[4], line[0]))

In [57]:
# SELECT for third query
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:
    results_3 = session.execute("SELECT first_name, last_name, song, user_id FROM music_library_third \
    WHERE song = 'All Hands Against His Own'")
except Exception as e:
    print(e)
    
for row in results_3:
    print(row.first_name, row.last_name, row.song, row.user_id)

Jacqueline Lynch All Hands Against His Own 29
Tegan Levine All Hands Against His Own 80
Sara Johnson All Hands Against His Own 95


### Drop the tables before closing out the sessions

In [66]:
## Drop table statements
try:
    session.execute("DROP TABLE IF EXISTS music_library_first")
    session.execute("DROP TABLE IF EXISTS music_library_second")
    session.execute("DROP TABLE IF EXISTS music_library_third")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [67]:
session.shutdown()
cluster.shutdown()